## 2020-09-30: Exploring Accuracy for Orthogonalization Algorithms

*Last Updated*: 2020-09-30

### Authors

* Kevin Chu (kevin@velexi.com)

### Overview

In this Jupyter notebook, we explore the loss of orthogonality of modified Gram-Schmidt (MGS) and Householder triangularization algorithms.

### Key Results

* __Loss of Orthogonality__. For "random matrices", both MGS and Householder triangularization produce $Q$ matrices where $Q' * Q - I$ is within a couple of orders of magnitude of machine precision.

    * $|Q' * Q - I|_2$ tends to be about an order of magnitude smaller for Householder triangularization than for MGS.
    
    * The standard deviation of $|Q' * Q - I|_2$ tends to be about several orders of magnitude smaller for Householder triangularization than for MGS.
    
* __Accuracy of Determinant__. For "random matrices", both MGS and Householder triangularization produce factorizations where the relative error of $\det(Q R)$ is close to machine precision.

    * The relative error of the $\det(Q R)$ is very comparable for both algorithms.
    
### Conclusions

* When orthogonality of $Q$ is important, Householder triangularization should be used to compute $Q$. For even simple and small matrices, MGS can lead to significant loss of orthogonality (see Special Cases section).

* When only the determinant of $A$ is important, both algorithms yield similar levels of accuracy.
    
### User parameters

* `n`: matrix size
* `num_samples`: number of matrices to sample

In [1]:
# --- User parameters

n = 100
num_samples = 10000

10000

In [2]:
# --- Imports

using BenchmarkTools
using LinearAlgebra
using Statistics

In [3]:
# --- Functions

"""
    Compute QR factorization of `A` using the modified Gram-Schmidt algorithm.
"""
function qr_mgs(A::Matrix)
    # Initialize Q and R
    Q = copy(A)
    R = zero(A)
    
    for j in 1:size(Q, 2)
        norm = LinearAlgebra.norm(Q[:, j])
        if norm > 0
            Q[:, j] /= norm
            R[j, j] = norm
        else
            continue
        end

        column = Q[:, j]
        for j_inner in (j + 1):size(Q, 2)
            R[j, j_inner] = Q[:, j_inner] ⋅ column
            Q[:, j_inner] -= R[j, j_inner] * column
        end
    end
    
    Q, R
end;

In [4]:
# --- Test qr_mgs()

n = 10
A = randn(n, n)
Q, R = qr_mgs(A)

println("Relative error Q * R: ", opnorm(A - Q * R)/opnorm(A))
println("Absolute error Q * R: ", opnorm(A - Q * R))
println("opnorm(Q' * Q): ", opnorm(transpose(Q) * Q))

det_A = det(A)
det_Q = det(Q)
det_R = det(R)
println("det(A): ", det_A)
println("det(Q): ", det_Q)
println("det(R): ", det_R)
println("Relative error det(Q * R):", (det_A - det_Q*det_R) / det_A)
println("Absolute error det(Q * R):", det_A - det_Q*det_R)

Relative error Q * R: 1.235430925418796e-16
Absolute error Q * R: 7.296305761975228e-16
opnorm(Q' * Q): 1.0000000000000004
det(A): -770.5772039632855
det(Q): -1.0000000000000002
det(R): 770.5772039632857
Relative error det(Q * R):-5.901385981152523e-16
Absolute error det(Q * R):4.547473508864641e-13


### Run Experiments

In [5]:
# Initialize data vectors
mgs_orthogonality_errors = zeros(num_samples)
householder_orthogonality_errors = zeros(num_samples)

mgs_det_errors = zeros(num_samples)
householder_det_errors = zeros(num_samples)

# Collect data
for i in 1:num_samples
    # Generate random matrix
    A = randn(n, n)
    det_A = det(A)

    # Compute QR factorization using modified Gram-Schmidt algorithm
    Q_mgs, R_mgs = qr_mgs(A)

    mgs_orthogonality_errors[i] = opnorm(transpose(Q_mgs) * Q_mgs - LinearAlgebra.I)

    det_R = det(R_mgs)
    mgs_det_errors[i] = abs((abs(det_A) - abs(det_R)) / det_A)
    
    # Compute QR factorization using Householder triangularization
    F_householder= qr(A)
    Q_householder = F_householder.Q
    R_householder = F_householder.R
    
    householder_orthogonality_errors[i] =
        opnorm(transpose(Q_householder) * Q_householder - LinearAlgebra.I)

    det_R = det(R_householder)
    householder_det_errors[i] = abs((abs(det_A) - abs(det_R)) / det_A)
end

### Results

In [6]:
# --- Orthogonality Error

println("mean(mgs_orthogonality_errors): ", mean(mgs_orthogonality_errors))
println("std(mgs_orthogonality_errors): ", std(mgs_orthogonality_errors))

println("mean(householder_orthogonality_errors): ", mean(householder_orthogonality_errors))
println("std(householder_orthogonality_errors): ", std(householder_orthogonality_errors))

mean(mgs_orthogonality_errors): 4.998935256305874e-14
std(mgs_orthogonality_errors): 3.900375254390148e-12
mean(householder_orthogonality_errors): 1.1448650345845443e-15
std(householder_orthogonality_errors): 2.8484152762388585e-16


In [7]:
# --- Determinant Error

println("mean(mgs_det_errors): ", mean(mgs_det_errors))
println("std(mgs_det_errors): ", std(mgs_det_errors))

println("mean(householder_det_errors): ", mean(householder_det_errors))
println("std(householder_det_errors): ", std(householder_det_errors))

mean(mgs_det_errors): 9.64652399768703e-15
std(mgs_det_errors): 6.672712064193565e-13
mean(householder_det_errors): 2.7625912189299647e-14
std(householder_det_errors): 2.2140359798021072e-12


### Special Cases

In [8]:
# --- Special cases that demonstrate large loss of orthogonality with MGS

A = [0.700000 0.70711; 0.70001 0.70711]
det_A = det(A)

# MGS
Q_mgs, R_mgs = qr_mgs(A)
println("MGS orthogonality error: ", opnorm(transpose(Q_mgs) * Q_mgs - LinearAlgebra.I))
println("MGS determinant error: ", abs((abs(det_A) - abs(det(R_mgs))) / det_A))

# Householder triangularization
F_householder = qr(A)
Q_householder = F_householder.Q
R_householder = F_householder.R
println("Householder triangularization orthogonality error: ",
        opnorm(transpose(Q_householder) * Q_householder - LinearAlgebra.I))
println("Householder triangularization determinant error: ",
        abs((abs(det_A) - abs(det(R_householder))) / det_A))

MGS orthogonality error: 2.3014368188967183e-11
MGS determinant error: 6.834983657002004e-12
Householder triangularization orthogonality error: 2.351490101248793e-16
Householder triangularization determinant error: 5.014565377055642e-12
